In [1]:
pwd

'/home/users/cdonoso/anshika/astromer/presentation/notebooks'

In [2]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import re
import zipfile
from io import BytesIO

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os
import sys,os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# This tutorial runs only in early mode. For the optimized code use train.py
# OTHERWISE you can comment the next ljupyine and reset the kernel before the finetuning
tf.config.run_functions_eagerly(False)

In [3]:
cd ../..

/home/users/cdonoso/anshika/astromer


In [4]:
from src.data.record  import create_dataset

In [5]:
source = 'g.zip' # lightcurves folder
metadata_path = "metadata_g_with_features.dat" # metadata file
target = 'records/new_ztf_g'

In [6]:
# filtering metadata

metadata_all = pd.read_table(metadata_path, sep=",")
b = metadata_all.Std_g>0.1 #basic filtering by std
metadata_filtered = metadata_all[b] #creating filtered version of metadata that is much smaller
metadata_filtered['Class'] = ['UNK']*(metadata_filtered.shape[0])
metadata_filtered['Band'] = tf.ones(metadata_filtered.shape[0])
metadata_filtered = metadata_filtered.rename(columns={'Path_g':'Path'})
print(metadata_filtered.head())
print(metadata_filtered.shape)

/tmp/ipykernel_45482/73907368.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_filtered['Class'] = ['UNK']*(metadata_filtered.shape[0])
2022-12-29 13:04:39.990759: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-29 13:04:39.990795: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: pececillo
2022-12-29 13:04:39.990801: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: pececillo
2022-12-29 13:04:39.990880: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 460.32.3
2022-12-29 13:04:39.990896: I tensorflow/stream_executor/cuda/cuda_d

                ID  N_g     Std_g  Skewness_g  Kurtosis_g   eta_e_g  \
0  350112100000000   63  0.252657   -0.595437    1.249473  0.469394   
2  350112100000004   69  0.130841   -0.224364    0.898845  2.969642   
4  350112100000006   41  0.220955   -0.055654   -0.662197  1.641803   
5  350112100000007   22  0.644202   -2.956395    9.948386  0.234716   
6  350112100000008  190  0.136506   -0.913373    2.359099  1.496204   

   cadence_g  FluxPercentile_g  \
0  18.543756          0.036768   
2  10.671661          0.021187   
4  29.547062          0.030162   
5  56.136309          0.062250   
6   6.278854          0.021298   

                                                Path Class  Band  
0  /home/AdditionalSpace2/Databases/ZTF/DR10/LCs/...   UNK   1.0  
2  /home/AdditionalSpace2/Databases/ZTF/DR10/LCs/...   UNK   1.0  
4  /home/AdditionalSpace2/Databases/ZTF/DR10/LCs/...   UNK   1.0  
5  /home/AdditionalSpace2/Databases/ZTF/DR10/LCs/...   UNK   1.0  
6  /home/AdditionalSpace2/Databas

In [7]:
# with zipfile.ZipFile('g.zip', 'r') as zp:
#     # files_info = zipfile.ZipFile.infolist(zp)
#     files_names = zipfile.ZipFile.namelist(zp)[1:]
    
# file_ids = list(map(lambda x: x.split('/')[-1].split('.')[0], files_names))
# # file_ids[:10]

In [8]:
# len(set(metadata_filtered.ID.astype(int).astype(str).values).intersection(set(file_ids)))

In [10]:
test_meta  = metadata_filtered.sample(frac=0.2)
train_meta = metadata_filtered[~metadata_filtered['ID'].isin(test_meta['ID'])]
create_dataset(train_meta, source, target, max_lcs_per_record=20000, 
               n_jobs=32, subsets_frac=(0.8, 0.2), test_subset=test_meta)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [1:29:10<00:00, 5350.27s/it]


In [ ]:
!ls records/new_ztf_g/train/UNK -f1 | head -7

In [ ]:
direc = "records/new_ztf_g/train/UNK"
count = 0
for filename in os.listdir(direc):
    if os.path.isfile(os.path.join(direc, filename)):
        count += 1
    #print(filename)
print(count)

In [ ]:
direc = "records/new_ztf_g/val/UNK"
count = 0
for filename in os.listdir(direc):
    if os.path.isfile(os.path.join(direc, filename)):
        count += 1
print(count)

In [ ]:
direc = "records/new_ztf_g/test/UNK"
count = 0
for filename in os.listdir(direc):
    if os.path.isfile(os.path.join(direc, filename)):
        count += 1
print(count)